In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/Users/sara/Documents/算法学习/NLP/nlp名企/MRC/')

In [2]:
import jieba
import pandas as pd
import numpy as np
from collections import defaultdict
from utils.data_loader import sentence_proc,sentences_proc,en_split_sentences_proc,documents_proc,split_sentences_proc,split_sentences_proc
from utils.file_utils import save_dict
from utils.multi_proc_utils import parallelize, cores
from utils.config import search_dev_data_path, zhidao_dev_data_path, merger_dev_seg_path,squad2_vocab_path,save_squad2_wv_model_path,squad_dev_data_path,squad2_merger_dev_seg_path,stop_word_path,save_wv_model_path,vocab_path,embedding_matrix_path,search_train_data_path, merger_seg_path,zhidao_train_data_path, search_test_data_path, zhidao_test_data_path,embedding_dim,wv_train_epochs
from gensim.models.word2vec import LineSentence, Word2Vec

In [6]:
# 训练 Dureader2.0

In [3]:
# 1.加载数据
search_dev_df = pd.read_json(search_dev_data_path, lines=True)
zhidao_dev_df = pd.read_json(zhidao_dev_data_path,encoding='utf-8', lines=True)
print('search dev data size {},zhidao dev data size {}'.format(len(search_dev_df), len(zhidao_dev_df)))

# 2、数据处理
search_dev_df['answers'] = search_dev_df[['answers']].apply(sentence_proc, axis=1)
search_dev_df['entity_answers'] = search_dev_df[['entity_answers']].apply(sentences_proc, axis=1)
search_dev_df['documents'] = search_dev_df[['documents']].apply(documents_proc, axis=1)
zhidao_dev_df['answers'] = zhidao_dev_df[['answers']].apply(sentence_proc, axis=1)
zhidao_dev_df['entity_answers'] = zhidao_dev_df[['entity_answers']].apply(sentences_proc, axis=1)
zhidao_dev_df['documents'] = zhidao_dev_df[['documents']].apply(documents_proc, axis=1)

# 3.多线程, 批量数据处理
search_dev_df = parallelize(search_dev_df, split_sentences_proc)
zhidao_dev_df = parallelize(zhidao_dev_df, split_sentences_proc)

# 4. 合并训练测试集合
search_dev_df['merged'] = search_dev_df[['documents', 'entity_answers', 'question', 'answers']].apply(lambda x: ' '.join(x), axis=1)
zhidao_dev_df['merged'] = search_dev_df[['documents', 'entity_answers', 'question', 'answers']].apply(lambda x: ' '.join(x), axis=1)
merged_df = pd.concat([search_dev_df[['merged']], zhidao_dev_df[['merged']]], axis=0)
print('train data size {},test data size {},merged_df data size {}'.format(len(search_dev_df),
                                                                           len(zhidao_dev_df),
                                                                           len(merged_df)))
# 6. 保存合并数据
merged_df.to_csv(merger_dev_seg_path, index=None, header=False)

# 7. 训练词向量
print('start build w2v model')
wv_model = Word2Vec(LineSentence(merger_dev_seg_path),
                    size=embedding_dim,
                    sg=1,
                    workers=cores,
                    iter=wv_train_epochs,
                    window=5,
                    min_count=5)
# 8. 填充开始结束符号,未知词填充 oov, 长度填充
# 使用GenSim训练得出的vocab
vocab = wv_model.wv.vocab

# 9、保存字典
save_dict(vocab_path, vocab)

# 10、保存词向量模型
wv_model.save(save_wv_model_path)
print('start build w2v model finished')

search dev data size 5000,zhidao dev data size 5000


Building prefix dict from the default dictionary ...
2020-09-20 09:15:36,075 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:15:36,090 : DEBUG : Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:15:36,126 : DEBUG : Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:15:36,137 : DEBUG : Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
Building prefix dict from the default dictionary ...
2020-09-20 09:15:36,187 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:15:36,198 : DEBUG : Loading model from cache /var/folders/49/6s8tc3j

train data size 5000,test data size 5000,merged_df data size 10000


2020-09-20 09:17:07,296 : INFO : collecting all words and their counts
2020-09-20 09:17:07,298 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


start build w2v model


2020-09-20 09:17:12,525 : INFO : PROGRESS: at sentence #10000, processed 16137801 words, keeping 265617 word types
2020-09-20 09:17:12,540 : INFO : collected 265617 word types from a corpus of 16181598 raw words and 10032 sentences
2020-09-20 09:17:12,541 : INFO : Loading a fresh vocabulary
2020-09-20 09:17:12,950 : INFO : effective_min_count=5 retains 114731 unique words (43% of original 265617, drops 150886)
2020-09-20 09:17:12,951 : INFO : effective_min_count=5 leaves 15796392 word corpus (97% of original 16181598, drops 385206)
2020-09-20 09:17:13,330 : INFO : deleting the raw counts dictionary of 265617 items
2020-09-20 09:17:13,338 : INFO : sample=0.001 downsamples 20 most-common words
2020-09-20 09:17:13,339 : INFO : downsampling leaves estimated 13672312 word corpus (86.6% of prior 15796392)
2020-09-20 09:17:13,709 : INFO : estimated required memory for 114731 words and 300 dimensions: 332719900 bytes
2020-09-20 09:17:13,710 : INFO : resetting layer weights
2020-09-20 09:17:35,

start build w2v model finished


In [10]:
# 训练 SQuAD2.0

In [3]:
# 1、读取数据
squad_dev_df = pd.read_json(squad_dev_data_path, encoding='utf-8', lines=True)
squad_dev_df = squad_dev_df['data']

# 2、数据处理
title = []
context = []
question = []
answer = []
merge = []
for datas in squad_dev_df:
    for data in datas:
        title.append(data['title'])
        for paragraph in data['paragraphs']:
            context.append(paragraph['context'])
            for qus in paragraph['qas']:
                question.append(qus['question'])
                for answers in  qus['answers']:
                    answer.append(answers['text'])

# 3、合并数据
merge = title + context + question + answer
squad_merge_df = pd.DataFrame(merge,columns=['merge']) 
print(squad_merge_df)
# 4.多线程, 批量数据处理
squad_merge_df = parallelize(squad_merge_df, en_split_sentences_proc)
print('merge_df data size {}'.format(len(squad_merge_df)))
print(squad_merge_df)
# 5. 保存合并数据
squad_merge_df.to_csv(squad2_merger_dev_seg_path, index=None, header=False)

# 6. 训练词向量
print('start build w2v model')
wv_model = Word2Vec(LineSentence(squad2_merger_dev_seg_path),
                    size=embedding_dim,
                    sg=1,
                    workers=cores,
                    iter=wv_train_epochs,
                    window=5,
                    min_count=5)

# 7、使用GenSim训练得出的vocab
vocab = wv_model.wv.vocab

# 8、保存字典
save_dict(squad2_vocab_path, vocab)

# 9、保存词向量模型
wv_model.save(save_squad2_wv_model_path)
print('start build w2v model finished')

                                 merge
0                              Normans
1      Computational_complexity_theory
2                  Southern_California
3                 Sky_(United_Kingdom)
4                 Victoria_(Australia)
...                                ...
33409                           sthène
33410                           sthène
33411                           sthène
33412                           sthène
33413                           sthène

[33414 rows x 1 columns]


Building prefix dict from the default dictionary ...
2020-09-20 09:28:11,075 : DEBUG : Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
2020-09-20 09:28:11,078 : DEBUG : Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:28:11,086 : DEBUG : Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:28:11,083 : DEBUG : Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
2020-09-20 09:28:11,090 : DEBUG : Loading model from cache /var/folders/49/6s8tc3jx6r94lyzs0dz49dr00000gn/T/jieba.cache
Building prefix dict from the

merge_df data size 33414
                                                   merge
0                                          N o r m a n s
1      C o m p u t a t i o n a l   _   c o m p l e x ...
2              S o u t h e r n   _   C a l i f o r n i a
3      S k y   _   (   U n i t e d   _   K i n g d o ...
4        V i c t o r i a   _   (   A u s t r a l i a   )
...                                                  ...
33409                                    s t h   è   n e
33410                                    s t h   è   n e
33411                                    s t h   è   n e
33412                                    s t h   è   n e
33413                                    s t h   è   n e

[33414 rows x 1 columns]


2020-09-20 09:28:16,539 : INFO : collecting all words and their counts
2020-09-20 09:28:16,540 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-09-20 09:28:16,724 : INFO : PROGRESS: at sentence #10000, processed 1256206 words, keeping 325 word types


start build w2v model


2020-09-20 09:28:16,778 : INFO : PROGRESS: at sentence #20000, processed 1521826 words, keeping 367 word types
2020-09-20 09:28:16,822 : INFO : PROGRESS: at sentence #30000, processed 1714101 words, keeping 387 word types
2020-09-20 09:28:16,837 : INFO : collected 388 word types from a corpus of 1778112 raw words and 33483 sentences
2020-09-20 09:28:16,838 : INFO : Loading a fresh vocabulary
2020-09-20 09:28:16,839 : INFO : effective_min_count=5 retains 243 unique words (62% of original 388, drops 145)
2020-09-20 09:28:16,840 : INFO : effective_min_count=5 leaves 1777878 word corpus (99% of original 1778112, drops 234)
2020-09-20 09:28:16,846 : INFO : deleting the raw counts dictionary of 388 items
2020-09-20 09:28:16,847 : INFO : sample=0.001 downsamples 30 most-common words
2020-09-20 09:28:16,848 : INFO : downsampling leaves estimated 415782 word corpus (23.4% of prior 1777878)
2020-09-20 09:28:16,849 : INFO : estimated required memory for 243 words and 300 dimensions: 704700 bytes


start build w2v model finished


In [8]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [9]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True